In [1]:
from datasets import load_dataset 
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import os
import time
import pandas as pd
from datasets import load_dataset 

dataset = load_dataset("NicolaiSivesind/human-vs-machine", "wiki_labeled")
dataset = dataset['test'].select(range(100))

/home/codespace/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
tokenizer = AutoTokenizer.from_pretrained("andreas122001/bloomz-3b-wiki-detector")
model = AutoModelForSequenceClassification.from_pretrained("andreas122001/bloomz-3b-wiki-detector")
model.to("cpu")
print()

Loading checkpoint shards: 100%|██████████| 2/2 [00:21<00:00, 10.57s/it]

In [3]:
quantized_model = torch.quantization.quantize_dynamic(model, {torch.nn.Linear}, dtype=torch.qint8)

In [4]:
def print_size_of_model(model):
    torch.save(model.state_dict(), "temp.p")
    print('Size (MB):', os.path.getsize("temp.p")/1e6)
    os.remove('temp.p')

print_size_of_model(model)
print_size_of_model(quantized_model)

Size (MB): 12010.384816
Size (MB): 4932.560034


In [5]:
tokenized_texts = [tokenizer(x['text'], padding=True, truncation=True, return_tensors="pt", max_length=512).to("cpu") for x in dataset]
time_taken = []
model = model.to("cpu")

for text in tokenized_texts:
    start_time = time.time()
    with torch.no_grad():
        logits = model(**text).logits
    end_time = time.time()
    time_taken.append(end_time - start_time)
output = pd.DataFrame({"times":time_taken})
output.to_csv("quantized_inference_time.csv", index=False)
display(output.describe())

,times
count,100.000000
mean,20.310679
std,10.191210
min,2.177129
25%,12.524602
50%,18.880404
75%,27.699589
max,50.635854
